In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from skimage.io import imread
from skimage.filters import threshold_li
from skimage.exposure import equalize_adapthist
from skimage.morphology import remove_small_objects, disk, ball, binary_erosion
from scipy.ndimage import median_filter
from sklearn.linear_model import LinearRegression

In [ ]:
# fig. 4 EMT volume
img = imread('/Volumes/davidh-ssd/chromemt_data/49801.tif')

# rough location of first patch
cut_siz = 94 # 94px ~ 120nm at 1.28nm pixel size
off = 0, 140, 40

cut = img[tuple((slice(o, o+cut_siz) for o in off))]

In [ ]:
def segment_like_paper(patch, clahe_size=78, min_object_size=500, radius=2):
    '''
    Segmentation pipeline similar to ChromEMT pipeline
    clahe_size is in pixels, 78 ~ 100nm at 1.28nm pixel size
    '''

    # 1) CLAHE
    patch_eq = equalize_adapthist(patch, clahe_size)
    
    # 2) Li thresholding
    mask = patch_eq < threshold_li(patch_eq)

    # 3) ImaheJ "Remove Outliers..." should correspond to median filter
    # TODO: planewise matches ImageJ more
    mask = median_filter(mask, footprint=ball(radius))

    # 4) remove small objects should correspond to size threshold in 3D Object Counter
    mask = remove_small_objects(mask, min_object_size)
    return mask

def continuous_erosion(mask, erosion_radius=None):
    '''
    Continuous erosion to estimate chromatin thickness

    Parameters
    ==========
    mask: binary array of ndim == 3
        the mask to erode
    erosion_radius: iterable of floats/ints (optional)
        radii at which to calculate residual chromatin volume after erosion
        defaults to integer pixel radii up to half the z size of mask

    Returns
    =======
    residual_volumes: float array with shape == (len(erosion_radius),)
        residual volume fractions
    '''

    # default radii: integer pixels
    if erosion_radius is None:
        erosion_radius = np.arange(1, mask.shape[0]//2)

    # TODO: slow, parallelize or iterative erosion with iterative application of small sphere
    eroded = np.stack([binary_erosion(mask, ball(r)) for r in erosion_radius])

    return eroded.sum(axis=(1,2,3)) / mask.sum()

def linear_fit_to_residual_volume(residual_volume, erosion_radius=None, n_first_values_to_include=5):
    '''
    Fit linear model erosion_radius ~ residual volume to residual volumes
    Returns
    =======
    estimated_diameter: float
        the estimated diameter (2x erosion_radius intercept / intersection of model with x-axis)
    model: LinearModel
        the fitted model
    '''

    # default radii: integer pixels
    if erosion_radius is None:
        erosion_radius = np.arange(1, residual_volume.size+1)

    residual_volume = residual_volume[:n_first_values_to_include]
    erosion_radius = erosion_radius[:n_first_values_to_include]

    lm = LinearRegression()
    lm.fit(residual_volume.reshape(-1, 1), erosion_radius)
    
    return lm.intercept_ * 2, lm



In [ ]:
# test linear fit
rv_test = np.concatenate([np.linspace(1,0.5,5),np.linspace(0.5, 0.1, 15)] ) + np.random.rayleigh(scale=0.1, size=20)
r, mod = linear_fit_to_residual_volume(rv_test)

# plot trace, fitted model line
plt.plot(rv_test)
plt.plot(mod.predict(np.linspace(1,0).reshape(-1,1))-1, np.linspace(1,0).reshape(-1,1))

In [ ]:
mask = segment_like_paper(cut)
erosion_radius=np.arange(1, 15)
residual_vol = continuous_erosion(mask, erosion_radius)

In [ ]:
d, mod = linear_fit_to_residual_volume(residual_vol, erosion_radius)

# plot residual volume fraction
plt.plot(erosion_radius, residual_vol)
# plot fitted line
plt.plot(mod.predict(np.linspace(residual_vol[0],0).reshape(-1,1)), np.linspace(residual_vol[0],0).reshape(-1,1))

# diameter in nm
d * 1.28

In [ ]:
fig, axs = plt.subplots(ncols=2)
axs[0].imshow(cut[0])
axs[1].imshow(segment_like_paper(cut)[0])

# chromatin volume fraction
segment_like_paper(cut).sum() / np.prod(cut.shape)

In [ ]:
def hypersphere(rank, radius):
    '''
    rank-n hypersphere, should be just like disk/ball
    '''
    return np.linalg.norm(np.stack(np.meshgrid(*[np.arange(-radius, radius+1)]*rank, indexing='ij')), axis=0)<=radius

patch_eq = equalize_adapthist(cut, 78)
mask = patch_eq < threshold_li(patch_eq)

fig, axs = plt.subplots(ncols=3, figsize=(10,5))
axs[0].imshow(patch_eq[0])
axs[1].imshow(mask[0])
axs[2].imshow(median_filter(mask[0], footprint=disk(2.5)))